In [1]:
import pandas as pd
pd.set_option('display.max_columns',None)
import tensorflow as tf
import pandas as pd
import numpy as np
import gc
import os 
import pickle
# 用于修正tensorflow不能复原问题
from tfdeterminism import patch
import random
patch()
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

TensorFlow version 2.0.0 has been patched using tfdeterminism version 0.3.0


In [2]:
gpu = tf.config.experimental.list_physical_devices(device_type='GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)
def set_all_seeds(rand_seed):
    np.random.seed(rand_seed)
    random.seed(rand_seed)
    os.environ['PYTHONHASHSEED'] = str(rand_seed)
    tf.random.set_seed(rand_seed)
set_all_seeds(42)

In [3]:
TRAIN_FILE = "train.csv"
TEST_FILE = "test.csv"
path='/data/wy/competition_learning/推荐算法项目/deepfm/data1/'
NUMERIC_COLS = [
    "ps_reg_01", "ps_reg_02", "ps_reg_03",
    "ps_car_12", "ps_car_13", "ps_car_14", "ps_car_15"
]

IGNORE_COLS = [
    "id", "target",
    "ps_calc_01", "ps_calc_02", "ps_calc_03", "ps_calc_04",
    "ps_calc_05", "ps_calc_06", "ps_calc_07", "ps_calc_08",
    "ps_calc_09", "ps_calc_10", "ps_calc_11", "ps_calc_12",
    "ps_calc_13", "ps_calc_14",
    "ps_calc_15_bin", "ps_calc_16_bin", "ps_calc_17_bin",
    "ps_calc_18_bin", "ps_calc_19_bin", "ps_calc_20_bin"
]

dfTrain = pd.read_csv(path+TRAIN_FILE)
dfTest = pd.read_csv(path+TEST_FILE)
df = pd.concat([dfTrain,dfTest],sort=True)

In [4]:
feature_dict = {}
total_feature = 0
for col in df.columns:
    if col in IGNORE_COLS:
        continue
    elif col in NUMERIC_COLS:
        feature_dict[col] = total_feature
        total_feature += 1
    else:
        unique_val = df[col].unique()
        feature_dict[col] = dict(zip(unique_val,range(total_feature,len(unique_val) + total_feature)))
        total_feature += len(unique_val)


In [5]:
train_y = dfTrain[['target']].values.tolist()
dfTrain.drop(['target','id'],axis=1,inplace=True)
train_feature_index = dfTrain.copy()
train_feature_value = dfTrain.copy()

for col in train_feature_index.columns:
    if col in IGNORE_COLS:
        train_feature_index.drop(col,axis=1,inplace=True)
        train_feature_value.drop(col,axis=1,inplace=True)
        continue
    elif col in NUMERIC_COLS:
        train_feature_index[col] = feature_dict[col]
    else:
        train_feature_index[col] = train_feature_index[col].map(feature_dict[col])
        train_feature_value[col] = 1

In [6]:
test_ids = dfTest['id'].values.tolist()
dfTest.drop(['id'],axis=1,inplace=True)

test_feature_index = dfTest.copy()
test_feature_value = dfTest.copy()

for col in test_feature_index.columns:
    if col in IGNORE_COLS:
        test_feature_index.drop(col,axis=1,inplace=True)
        test_feature_value.drop(col,axis=1,inplace=True)
        continue
    elif col in NUMERIC_COLS:
        test_feature_index[col] = feature_dict[col]
    else:
        test_feature_index[col] = test_feature_index[col].map(feature_dict[col])
        test_feature_value[col] = 1

In [128]:
# 参数
emb_size=32
fea_size=total_feature
field_size=len(train_feature_index.columns)
fcn=[32,32]
learn_rate=0.001
epoch=30

In [131]:
class deepfm(tf.keras.Model):
    def __init__(self,emb_size,fea_size,field_size):
        super(deepfm,self).__init__()
        self.emb_fea=tf.keras.layers.Embedding(fea_size,emb_size)
        self.emb_bias=tf.keras.layers.Embedding(fea_size,1)
        self.fcn1=tf.keras.layers.Dense(64,activation='relu')
        self.fcn2=tf.keras.layers.Dense(32,activation='relu')
        self.fcn3=tf.keras.layers.Dense(1,activation='sigmoid')
        
    def call(self,inputs):
        fea_ind=inputs[0]
        fea_val=inputs[1]
        embs_fea_ind=self.emb_fea(fea_ind)
        fea_val_tf=tf.reshape(fea_val,[-1,field_size,1])
        # 共用embbeding层
        # [B,F,K]
        embeddings=embs_fea_ind*fea_val_tf
        
        # fm
        # 一次项
        fm_first=self.emb_bias(fea_ind)
        fea_val_fm=tf.expand_dims(fea_val,axis=2)
        fm_first=tf.reduce_sum((fm_first*fea_val_tf),1)
        
        # 二次项
        sum_squ=tf.square(tf.reduce_sum(embeddings,1))
        squ_sum=tf.reduce_sum(tf.square(embeddings),1)
        fm_second=(sum_squ-squ_sum)*0.5
        
        #fcn 
        fcn1=self.fcn1(embeddings)
        fcn2=self.fcn2(fcn1)
        fcn2=tf.keras.layers.Flatten()(fcn2)
        output=tf.concat([fm_first,fm_second,fcn2],axis=1)
        output=self.fcn3(output)
        return output

In [132]:
model=deepfm(emb_size,fea_size,field_size)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy')
model.fit(x=(train_feature_index.values,train_feature_value.values),y=np.array(train_y),epochs=10,batch_size=1024,validation_split=0.3)

Train on 416648 samples, validate on 178564 samples
Epoch 1/10
416648/416648 [==============================] - 3s 8us/sample - loss: 0.1728 - val_loss: 0.1533
Epoch 2/10
416648/416648 [==============================] - 3s 8us/sample - loss: 0.1523 - val_loss: 0.1529
Epoch 3/10
416648/416648 [==============================] - 4s 8us/sample - loss: 0.1521 - val_loss: 0.1529
Epoch 4/10
416648/416648 [==============================] - 3s 8us/sample - loss: 0.1519 - val_loss: 0.1546
Epoch 5/10
416648/416648 [==============================] - 4s 8us/sample - loss: 0.1518 - val_loss: 0.1527
Epoch 6/10
416648/416648 [==============================] - 4s 8us/sample - loss: 0.1516 - val_loss: 0.1530
Epoch 7/10
416648/416648 [==============================] - 3s 8us/sample - loss: 0.1514 - val_loss: 0.1529
Epoch 8/10
416648/416648 [==============================] - 3s 8us/sample - loss: 0.1510 - val_loss: 0.1530
Epoch 9/10
416648/416648 [==============================] - 3s 8us/sample - loss: 0.